In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
# 銘柄の一覧
url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
meigara_df = pd.read_excel(url)
all_tickers = [str(row['コード'])+'.T' for i, row in meigara_df.iterrows()]

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

def fetch_stock_data(ticker):
    """指定された株式のデータを取得"""
    stock = yf.Ticker(ticker)
    
    # 財務情報の取得
    info = stock.info
    financials = stock.financials
    balance_sheet = stock.balance_sheet
    cashflow = stock.cashflow
    
    return {
        'info': info,
        'financials': financials,
        'balance_sheet': balance_sheet,
        'cashflow': cashflow
    }

def calculate_financial_score(data):
    """財務指標に基づくスコアを計算"""
    try:
        # 直近の売上高と純利益
        revenue = data['financials'].loc['Total Revenue'].iloc[0]
        net_income = data['financials'].loc['Net Income'].iloc[0]
        
        # 利益率計算
        profit_margin = net_income / revenue * 100
        
        # スコア化（0-100）
        financial_score = min(max(profit_margin, 0), 100)
        return financial_score
    except:
        return 50  # エラー時のデフォルトスコア

def calculate_growth_score(data):
    """成長性スコアを計算"""
    try:
        # 過去3年間の売上高成長率
        revenues = data['financials'].loc['Total Revenue']
        growth_rates = [(revenues.iloc[i] - revenues.iloc[i+1]) / revenues.iloc[i+1] * 100 for i in range(len(revenues)-1)]
        avg_growth = np.mean(growth_rates)
        
        # スコア化（0-100）
        growth_score = min(max(avg_growth + 50, 0), 100)
        return growth_score
    except:
        return 50  # エラー時のデフォルトスコア

def calculate_stability_score(data):
    """安定性スコアを計算"""
    try:
        # 負債比率と流動性の確認
        total_assets = data['balance_sheet'].loc['Total Assets'].iloc[0]
        total_liabilities = data['balance_sheet'].loc['Total Liabilities Net Minority Interest'].iloc[0]
        debt_ratio = total_liabilities / total_assets * 100
        
        # キャッシュフローの安定性
        operating_cashflows = data['cashflow'].loc['Operating Cash Flow']
        cashflow_std = np.std(operating_cashflows)
        cashflow_mean = np.mean(operating_cashflows)
        cashflow_variation = cashflow_std / abs(cashflow_mean) * 100
        
        # スコア化（低い負債比率と低い変動性が高スコア）
        stability_score = 100 - min(debt_ratio, 50) - min(cashflow_variation, 50)
        return max(stability_score, 0)
    except:
        return 50  # エラー時のデフォルトスコア

def calculate_valuation_score(data):
    """バリュエーションスコアを計算"""
    try:
        # PER（株価収益率）とPBR（株価純資産倍率）
        pe_ratio = data['info'].get('trailingPE', 0)
        pb_ratio = data['info'].get('priceToBook', 0)
        
        # 理想的な範囲内かを評価
        pe_score = max(100 - abs(pe_ratio - 15), 0)  # 15が理想的なPER
        pb_score = max(100 - abs(pb_ratio - 1), 0)   # 1が理想的なPBR
        
        valuation_score = (pe_score + pb_score) / 2
        return valuation_score
    except:
        return 50  # エラー時のデフォルトスコア

def calculate_total_score(ticker):
    """総合スコアの計算"""
    try:
        # 株式データの取得
        stock_data = fetch_stock_data(ticker)
        
        # 各スコアの計算
        financial_score = calculate_financial_score(stock_data)
        growth_score = calculate_growth_score(stock_data)
        stability_score = calculate_stability_score(stock_data)
        valuation_score = calculate_valuation_score(stock_data)
        
        # 重み付けして総合スコアを計算
        total_score = (
            financial_score * 0.3 +
            growth_score * 0.25 +
            stability_score * 0.25 +
            valuation_score * 0.2
        )
        
        return {
            'Ticker': ticker,
            'Financial Score': round(financial_score, 2),
            'Growth Score': round(growth_score, 2),
            'Stability Score': round(stability_score, 2),
            'Valuation Score': round(valuation_score, 2),
            'Total Score': round(total_score, 2)
        }
    except Exception as e:
        print(f"エラー: {e}")
        return None

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

def parallel_calculate_scores(all_tickers):
    results = []
    
    # ProcessPoolExecutor を使用して並列処理
    with ProcessPoolExecutor() as executor:
        # future オブジェクトを作成
        futures = {executor.submit(calculate_total_score, ticker): ticker for ticker in all_tickers}
        
        # tqdm を使用して進捗を表示
        with tqdm(total=len(futures), desc="Processing Tickers", unit="ticker") as pbar:
            for future in as_completed(futures):
                ticker = futures[future]
                try:
                    result = future.result()
                    results.append(result)
                    pbar.update(1)
                except Exception as exc:
                    print(f'{ticker} generated an exception: {exc}')
    
    return results

# 使用例
results = parallel_calculate_scores(all_tickers)

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df.sort_values('Total Score', ascending=False).to_excel('result.xlsx')